In [1]:
import torch, sys
print("Python:", sys.version)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch: 2.8.0+cu126
CUDA available: False
Device: CPU


In [2]:
!pip uninstall -y transformers accelerate tokenizers -q
!pip install -U "transformers==4.46.1" "accelerate>=0.34.2" "datasets>=2.20.0" "evaluate>=0.4.2" scikit-learn -q


In [3]:
import IPython, os
if not os.environ.get("RESTARTED_COLAB"):
    os.environ["RESTARTED_COLAB"] = "1"
    IPython.display.display(IPython.display.HTML("<h3>🔁 Restarting...</h3>"))
    IPython.Application.instance().kernel.do_shutdown(restart=True)


In [1]:
import random, numpy as np, torch, os
from datasets import load_dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, Trainer, TrainingArguments)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import pandas as pd

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

# Labels for UKPLab/liar: 0=REAL, 1=FAKE
ID2LABEL = {0: "REAL", 1: "FAKE"}
LABEL2ID = {v:k for k,v in ID2LABEL.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    # Treat FAKE as positive (pos_label=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", pos_label=1)
    return {"accuracy": acc, "precision_fake": precision, "recall_fake": recall, "f1_fake": f1}


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
dataset = load_dataset("UKPLab/liar")  # splits: train/validation/test
print("Columns:", dataset["train"].column_names)  # -> ['text','label_text','labels','context']

# Sanity: label distribution
from collections import Counter
for split in ["train", "validation", "test"]:
    print(split, Counter(dataset[split]["labels"]))
print("Example:", dataset["train"][0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


Columns: ['text', 'label_text', 'labels', 'context']
train Counter({0: 7429, 1: 2840})
validation Counter({0: 905, 1: 379})
test Counter({0: 941, 1: 342})
Example: {'text': 'Dwayne Bohac says the Annies List political group supports third-trimester abortions on demand.', 'label_text': 'false statement', 'labels': 1, 'context': 'a mailer'}


In [3]:
MODEL_NAME = "facebook/bart-base"
TEXT_FIELD = "text"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess(batch):
    return tokenizer(batch[TEXT_FIELD], truncation=True, max_length=256)

cols_to_remove = [c for c in dataset["train"].column_names if c not in [TEXT_FIELD, "labels"]]
tokenized = dataset.map(preprocess, batched=True, remove_columns=cols_to_remove)

for split in ["train","validation","test"]:
    print(split, tokenized[split])


Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

train Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 10269
})
validation Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1284
})
test Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1283
})


In [4]:
data_collator = DataCollatorWithPadding(tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    id2label=ID2LABEL,
    label2id=LABEL2ID,
)

BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
WEIGHT_DECAY = 0.01

training_args = TrainingArguments(
    output_dir="/content/bart-liar-bin",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY,
    load_best_model_at_end=True,
    metric_for_best_model="f1_fake",
    logging_steps=50,
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipython-input-1702030002.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
train_result = trainer.train()
train_result


In [9]:
metrics_val = trainer.evaluate(tokenized["validation"])
metrics_test = trainer.evaluate(tokenized["test"])
print("Validation:", metrics_val)
print("Test:", metrics_test)

# Confusion matrix on test
preds = trainer.predict(tokenized["test"])
y_true = preds.label_ids
y_pred = preds.predictions.argmax(-1)
cm = confusion_matrix(y_true, y_pred, labels=[0,1])  # rows: REAL, FAKE
cm_df = pd.DataFrame(cm, index=["REAL_true","FAKE_true"], columns=["REAL_pred","FAKE_pred"])
cm_df


Transformers: 4.57.1
Datasets: 4.2.0
Accelerate: 1.10.1
Evaluate: 0.4.6


In [9]:
SAVE_DIR = "/content/models/bart-base-liar-bin"
os.makedirs(SAVE_DIR, exist_ok=True)
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
!ls -lah "$SAVE_DIR"


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, json

id2label = {0:"REAL", 1:"FAKE"}
tok = AutoTokenizer.from_pretrained(SAVE_DIR)
mdl = AutoModelForSequenceClassification.from_pretrained(SAVE_DIR).eval()

def predict(text, max_length=256):
    enc = tok(text, return_tensors="pt", truncation=True, max_length=max_length)
    with torch.no_grad():
        out = mdl(**enc)
        probs = out.logits.softmax(-1).squeeze().tolist()
        pred = int(out.logits.argmax(-1))
    return {"label": id2label[pred], "probs": {"REAL": probs[0], "FAKE": probs[1]}}

samples = [
    "Breaking: Aliens land in Berlin—official sources confirm.",
    "The Ministry of Health published new vaccination guidance today."
]
for s in samples:
    print(s, "→", json.dumps(predict(s), indent=2))
